In [ ]:
import h5py
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 as cv
#print(os.listdir('./data/'))
noExamples = len(os.listdir('./data'))
data = np.zeros((noExamples,(224*224)+1,1))
labels = np.zeros((noExamples,1))
for itemId, file in zip(range(0,noExamples),os.listdir('./data')):
    with h5py.File('./data/'+file, 'r') as f:
        tmpdata = np.array(f['cjdata']['image'])
        data[itemId,0:224*224,:] = cv.resize(tmpdata, (224,224)).reshape(224*224,1)
        data[itemId,224*224,:] = f['cjdata']['label']
        print(itemId)

        
#20% for Test, 80% for Training


In [8]:

noTrain=int(0.8*noExamples);
print(data.shape[0])
#Random Splitting of Data
train,test,__ = np.vsplit(data[np.random.permutation(data.shape[0])],(noTrain,noExamples))
xtrain = data[0:noTrain,0:224*224,:].reshape(noTrain,224,224,1)
y_train = data[0:noTrain,224*224,:].reshape(noTrain) 
y_train = y_train.astype(int)
xtest = data[noTrain:noExamples,0:224*224,:].reshape(noExamples-noTrain,224,224,1)
y_test = data[noTrain:noExamples,224*224,:].reshape(noExamples-noTrain);
y_test = y_test.astype(int)
x_train, x_test = xtrain/255, xtest/255

#print(xtrain.shape)

3064


In [ ]:
#Simple Splitting
#x_train,x_test = data[0:noTrain,:,:,:]/255,data[noTrain:noExamples:,:,:,:]/255
#y_train,y_test = labels[0:noTrain], labels[noTrain:noExamples:]


In [10]:
 model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=48,kernel_size=11,strides=(2,2),padding="valid", activation='relu', input_shape=(224,224,1)), 
  tf.keras.layers.MaxPool2D((2,2),padding="valid"),
  tf.keras.layers.Conv2D(filters=128,kernel_size=5,strides=(2,2),padding="valid", activation='relu'), 
  tf.keras.layers.MaxPool2D((2,2),padding="valid"),
  tf.keras.layers.Conv2D(filters=192,kernel_size=3,padding="valid", activation='relu'),    
  tf.keras.layers.Conv2D(filters=192,kernel_size=3,padding="valid", activation='relu'),
  tf.keras.layers.Conv2D(filters=128,kernel_size=3,padding="valid", activation='relu'),         
  tf.keras.layers.MaxPool2D((2,2),padding="valid"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(2048, activation='tanh'),
  tf.keras.layers.Dense(2048, activation='tanh'),
  tf.keras.layers.Dense(4, activation=tf.nn.softmax),
 ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

#print(data[0,200:210,200:210])
#plt.imshow(data[0,:,:])
#plt.show()
#print(data.shape)
#print(labels.shape)
#print(labels)


Epoch 1/5
2451/2451 [==============================] - 50s 20ms/sample - loss: 0.9087 - accuracy: 0.6932
Epoch 2/5
2451/2451 [==============================] - 52s 21ms/sample - loss: 0.4247 - accuracy: 0.8221
Epoch 3/5
2451/2451 [==============================] - 51s 21ms/sample - loss: 0.2256 - accuracy: 0.9208
Epoch 4/5
2451/2451 [==============================] - 54s 22ms/sample - loss: 0.1719 - accuracy: 0.9421
Epoch 5/5
2451/2451 [==============================] - 52s 21ms/sample - loss: 0.1241 - accuracy: 0.9580


In [12]:
model.evaluate(x_test, y_test)

613/613 [==============================] - 4s 6ms/sample - loss: 2.3553 - accuracy: 0.3752


[2.355336336093859, 0.3752039]